# Further improving the simple net in TensorFlow with dropout

We decide to randomly drop – with the `DROPOUT` probability – some of
the values propagated inside our internal dense network of hidden layers during training. In machine
learning this is a well-known form of regularization. Surprisingly enough, this idea of randomly
dropping a few values can improve our performance. The idea behind this improvement is that random
dropouts force the network to learn redundant patterns that are useful for better generalization:

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

2023-07-30 14:54:39.415679: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-30 14:54:40.865400: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Network and training.
EPOCHS = 200
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10 # number of outputs = number of digits
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2 # how much TRAIN is reserved for VALIDATION
DROPOUT = 0.3

In [3]:
# Loading MNIST dataset
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

In [4]:
# X_train is 60000 rows of 28x28 values; we reshape it to 60000 x 784
RESHAPED = 784

X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [5]:
# Normalize inputs within [0, 1]
X_train, X_test = X_train / 255.0, X_test / 255.0
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

60000 train samples
10000 test samples


In [6]:
# One-hot representations for labels
Y_train = keras.utils.to_categorical(Y_train, NB_CLASSES)
Y_test = keras.utils.to_categorical(Y_test, NB_CLASSES)

In [7]:
# Building the model
model = tf.keras.models.Sequential()
model.add(keras.layers.Dense(N_HIDDEN,
                             input_shape=(RESHAPED,),
                             name='dense_layer',
                             activation='relu'))
model.add(keras.layers.Dropout(DROPOUT))
model.add(keras.layers.Dense(N_HIDDEN,
                             name='dense_layer_2',
                             activation='relu'))
model.add(keras.layers.Dropout(DROPOUT))
model.add(keras.layers.Dense(NB_CLASSES,
                             name='dense_layer_3',
                             activation='softmax'))

2023-07-30 14:54:44.193988: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1617 MB memory:  -> device: 0, name: NVIDIA GeForce MX130, pci bus id: 0000:01:00.0, compute capability: 5.0


In [8]:
# Summary of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_layer (Dense)         (None, 128)               100480    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_layer_2 (Dense)       (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_layer_3 (Dense)       (None, 10)                1290      
                                                                 
Total params: 118282 (462.04 KB)
Trainable params: 118282 (462.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
# Compiling the model
model.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
# Training the model
model.fit(X_train, Y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=VERBOSE,
          validation_split=VALIDATION_SPLIT)

2023-07-30 14:54:44.470856: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 150528000 exceeds 10% of free system memory.
2023-07-30 14:54:44.818396: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 150528000 exceeds 10% of free system memory.


Epoch 1/200


2023-07-30 14:54:46.734726: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f41faa65970 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-30 14:54:46.734767: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce MX130, Compute Capability 5.0
2023-07-30 14:54:47.062673: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8903
2023-07-30 14:54:47.306104: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


371/375 [============================>.] - ETA: 0s - loss: 1.6818 - accuracy: 0.4704

2023-07-30 14:54:50.711502: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 37632000 exceeds 10% of free system memory.
2023-07-30 14:54:50.806256: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 37632000 exceeds 10% of free system memory.


375/375 [==============================] - 6s 10ms/step - loss: 1.6763 - accuracy: 0.4721 - val_loss: 0.9057 - val_accuracy: 0.8129
Epoch 2/200
375/375 [==============================] - 3s 8ms/step - loss: 0.9242 - accuracy: 0.7129 - val_loss: 0.5420 - val_accuracy: 0.8641
Epoch 3/200
375/375 [==============================] - 3s 8ms/step - loss: 0.7044 - accuracy: 0.7810 - val_loss: 0.4335 - val_accuracy: 0.8842
Epoch 4/200
375/375 [==============================] - 3s 8ms/step - loss: 0.6004 - accuracy: 0.8162 - val_loss: 0.3766 - val_accuracy: 0.8979
Epoch 5/200
375/375 [==============================] - 3s 9ms/step - loss: 0.5355 - accuracy: 0.8397 - val_loss: 0.3412 - val_accuracy: 0.9059
Epoch 6/200
375/375 [==============================] - 2s 6ms/step - loss: 0.4950 - accuracy: 0.8512 - val_loss: 0.3180 - val_accuracy: 0.9103
Epoch 7/200
375/375 [==============================] - 2s 6ms/step - loss: 0.4622 - accuracy: 0.8610 - val_loss: 0.2986 - val_accuracy: 0.9150
Epoch 8/20

375/375 [==============================] - 2s 5ms/step - loss: 0.1716 - accuracy: 0.9489 - val_loss: 0.1200 - val_accuracy: 0.9628
Epoch 59/200
375/375 [==============================] - 2s 5ms/step - loss: 0.1669 - accuracy: 0.9514 - val_loss: 0.1194 - val_accuracy: 0.9637
Epoch 60/200
375/375 [==============================] - 2s 5ms/step - loss: 0.1669 - accuracy: 0.9505 - val_loss: 0.1179 - val_accuracy: 0.9639
Epoch 61/200
375/375 [==============================] - 2s 5ms/step - loss: 0.1653 - accuracy: 0.9511 - val_loss: 0.1174 - val_accuracy: 0.9639
Epoch 62/200
375/375 [==============================] - 2s 5ms/step - loss: 0.1644 - accuracy: 0.9518 - val_loss: 0.1169 - val_accuracy: 0.9643
Epoch 63/200
375/375 [==============================] - 2s 5ms/step - loss: 0.1635 - accuracy: 0.9523 - val_loss: 0.1157 - val_accuracy: 0.9645
Epoch 64/200
375/375 [==============================] - 2s 5ms/step - loss: 0.1587 - accuracy: 0.9530 - val_loss: 0.1145 - val_accuracy: 0.9651
Epoch

Epoch 115/200
375/375 [==============================] - 2s 5ms/step - loss: 0.1164 - accuracy: 0.9651 - val_loss: 0.0917 - val_accuracy: 0.9711
Epoch 116/200
375/375 [==============================] - 2s 5ms/step - loss: 0.1115 - accuracy: 0.9667 - val_loss: 0.0909 - val_accuracy: 0.9723
Epoch 117/200
375/375 [==============================] - 2s 5ms/step - loss: 0.1119 - accuracy: 0.9661 - val_loss: 0.0914 - val_accuracy: 0.9724
Epoch 118/200
375/375 [==============================] - 2s 5ms/step - loss: 0.1095 - accuracy: 0.9671 - val_loss: 0.0913 - val_accuracy: 0.9722
Epoch 119/200
375/375 [==============================] - 2s 5ms/step - loss: 0.1093 - accuracy: 0.9661 - val_loss: 0.0904 - val_accuracy: 0.9718
Epoch 120/200
375/375 [==============================] - 2s 5ms/step - loss: 0.1119 - accuracy: 0.9662 - val_loss: 0.0904 - val_accuracy: 0.9727
Epoch 121/200
375/375 [==============================] - 2s 5ms/step - loss: 0.1082 - accuracy: 0.9679 - val_loss: 0.0901 - val_ac

375/375 [==============================] - 2s 5ms/step - loss: 0.0871 - accuracy: 0.9739 - val_loss: 0.0827 - val_accuracy: 0.9753
Epoch 172/200
375/375 [==============================] - 2s 5ms/step - loss: 0.0878 - accuracy: 0.9725 - val_loss: 0.0819 - val_accuracy: 0.9750
Epoch 173/200
375/375 [==============================] - 2s 5ms/step - loss: 0.0873 - accuracy: 0.9729 - val_loss: 0.0821 - val_accuracy: 0.9747
Epoch 174/200
375/375 [==============================] - 2s 5ms/step - loss: 0.0847 - accuracy: 0.9737 - val_loss: 0.0826 - val_accuracy: 0.9751
Epoch 175/200
375/375 [==============================] - 2s 5ms/step - loss: 0.0822 - accuracy: 0.9745 - val_loss: 0.0820 - val_accuracy: 0.9747
Epoch 176/200
375/375 [==============================] - 2s 5ms/step - loss: 0.0856 - accuracy: 0.9738 - val_loss: 0.0819 - val_accuracy: 0.9752
Epoch 177/200
375/375 [==============================] - 2s 5ms/step - loss: 0.0830 - accuracy: 0.9755 - val_loss: 0.0814 - val_accuracy: 0.9751

In [11]:
# Evaluating the model
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy:', test_acccc)

 18/313 [>.............................] - ETA: 0s - loss: 0.0518 - accuracy: 0.9844 

2023-07-30 15:01:47.676215: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 31360000 exceeds 10% of free system memory.


313/313 [==============================] - 1s 3ms/step - loss: 0.0738 - accuracy: 0.9773
Test accuracy: 0.9772999882698059
